## N-Queens problem

In [51]:
import numpy as np

In [52]:
class Queen:
    def __init__(self, N):
        self.N = N
        self.queen_loc = dict()
        self.initialize = False
        self.chess_board = [[0] * self.N for _ in range(self.N)]

    def add_queen(self):
        if self.initialize == False:
            number_Q = 0
            while True:
                flag = 0
                r = np.random.randint(self.N)
                c = np.random.randint(self.N)
                #print(f'r:{r} c:{c}')
                for q in self.queen_loc:
                    row, col = self.queen_loc[q]
                    if (r == row and c == col) or (c == col):
                        flag = 1
                if flag == 0:
                    Q = f"Q{number_Q}"
                    if Q not in self.queen_loc:
                        self.queen_loc[Q] = []
                    self.queen_loc[Q].append(r)
                    self.queen_loc[Q].append(c)
                    self.chess_board[r][c] = Q
                    number_Q += 1
                if number_Q == self.N:
                    break
            self.initialize = True

    def get_neighbor(self, row, col):
        neighbor = []
        #print('Hello')
        if 0 <= row - 1 < self.N and self.chess_board[row - 1][col] == 0:
            neighbor.append([row - 1, col])
        if 0 <= row + 1 < self.N and self.chess_board[row + 1][col] == 0:
            neighbor.append([row + 1, col])
        return neighbor

    def print_Queen(self):
        print(self.chess_board)
        for Q in self.queen_loc:
            print(f'{Q}->{self.queen_loc[Q]}')

    # Steepest ascent hill climbing implementation

    def steepest_ascent_hill_climbing(self):
        current_state = self.queen_loc.copy()
        current_cost, max_cost, maxQ = calc_cost(current_state)

        while not goal_test(current_state):
            best_neighbor = None
            best_neighbor_cost = float('inf')

            for Q in current_state:
                row, col = current_state[Q]
                neighbors = self.get_neighbor(row, col)

                for neighbor_row, neighbor_col in neighbors:
                    neighbor_state = current_state.copy()
                    neighbor_state[Q] = [neighbor_row, neighbor_col]

                    neighbor_cost, neighbor_max_cost, _ = calc_cost(neighbor_state)

                    if neighbor_max_cost < best_neighbor_cost:
                        best_neighbor = neighbor_state
                        best_neighbor_cost = neighbor_max_cost

            if best_neighbor_cost >= max_cost:
                break

            current_state = best_neighbor
            current_cost, max_cost, maxQ = calc_cost(current_state)

        self.queen_loc = current_state
        self.update_chess_board()

    def update_chess_board(self):
        self.chess_board = [[0] * self.N for _ in range(self.N)]
        for Q in self.queen_loc:
            row, col = self.queen_loc[Q]
            self.chess_board[row][col] = Q


    # Simulated Annealing Implementation

    def simulated_annealing(self, initial_temperature=1000, cooling_rate=0.95):
        current_state = self.queen_loc.copy()
        current_cost, max_cost, maxQ = calc_cost(current_state)

        temperature = initial_temperature

        while temperature > 0:
            if current_cost == 0:
                break

            row, col = current_state[maxQ]
            neighbors = self.get_neighbor(row, col)

            if neighbors:
                neighbor_row, neighbor_col = np.random.choice(np.array(neighbors).flatten())
                neighbor_state = current_state.copy()
                neighbor_state[maxQ] = [neighbor_row, neighbor_col]

                neighbor_cost, neighbor_max_cost, _ = calc_cost(neighbor_state)

                cost_difference = neighbor_max_cost - max_cost

                if cost_difference <= 0 or np.random.uniform(0, 1) < np.exp(-cost_difference / temperature):
                    current_state = neighbor_state
                    current_cost = neighbor_cost
                    max_cost = neighbor_max_cost

            temperature *= cooling_rate

        self.queen_loc = current_state
        self.update_chess_board()


In [53]:
def conflict(r1, c1, r2, c2):
    if r1 == r2:
        return True
    if c1 == c2:
        return True
    if r1 + c1 == r2 + c2:
        return True
    if r1 - c1 == r2 - c2:
        return True
    return False

In [54]:
def get_conflict(Q, state):
    count = 0
    for q in state:
        if q is not Q:
            r1, c1 = state[Q]
            r2, c2 = state[q]
            if conflict(r1, c1, r2, c2):
                count += 1
    return count

In [55]:
def calc_cost(state):
    cost = 0
    max = -999
    maxQ = None
    #print(state)
    for Q in state:
        q_cost = get_conflict(Q, state)
        cost += q_cost
        if q_cost > max:
            max = q_cost
            maxQ = Q
    return cost // 2, max, maxQ

In [56]:
def goal_test(state):
    if calc_cost(state)[0] == 0:
        return True
    else:
        return False

In [57]:
queen = Queen(4)
queen.add_queen()
queen.print_Queen()

print("Running Steepest Ascent Hill Climbing:")
queen.steepest_ascent_hill_climbing()
queen.print_Queen()

print("Running Simulated Annealing:")
queen.simulated_annealing()
queen.print_Queen()

[[0, 'Q1', 0, 0], [0, 0, 0, 0], [0, 0, 0, 'Q2'], ['Q3', 0, 'Q0', 0]]
Q0->[3, 2]
Q1->[0, 1]
Q2->[2, 3]
Q3->[3, 0]
Running Steepest Ascent Hill Climbing:
[[0, 'Q1', 0, 0], [0, 0, 0, 'Q2'], ['Q3', 0, 0, 0], [0, 0, 'Q0', 0]]
Q0->[3, 2]
Q1->[0, 1]
Q2->[1, 3]
Q3->[2, 0]
Running Simulated Annealing:
[[0, 'Q1', 0, 0], [0, 0, 0, 'Q2'], ['Q3', 0, 0, 0], [0, 0, 'Q0', 0]]
Q0->[3, 2]
Q1->[0, 1]
Q2->[1, 3]
Q3->[2, 0]


In [58]:
# Simulated annealing giving "cannot unpack non-iterable numpy.int32 object" error sometimes, please check multiple times